In [1]:
import gzip
from astropy.table import Table

In [2]:
column_info_fname = "/Users/aphearin/work/sims/bolplanck/halo_catalogs/halotools_v0p4/column_info.dat"
column_info_table = Table.read(column_info_fname, format='ascii.commented_header')

In [7]:
halocat_fname = "/Users/aphearin/work/sims/bolplanck/halo_catalogs/halotools_v0p4/hlist_0.08037.list.gz"

import os

output_base_dirname = os.path.dirname(halocat_fname)
halocat_basename = os.path.basename(halocat_fname)
try:
    # sanity check on string formatting
    first_idx, last_idx = len('hlist_'), -len('.list.gz')
    a = float(halocat_basename[first_idx:last_idx])
    assert a > 0.
except:
    raise ValueError("halocat_fname basename {0} not formatted as expected".format(halocat_basename))

a_substring = 'a' + halocat_basename[first_idx-1: last_idx]
# print(a_substring)
output_binaries_dirname = os.path.join(output_base_dirname, a_substring)
print("passively creating the output dirname = \n    {0}".format(output_binaries_dirname))
try:
    os.makedirs(output_binaries_dirname)
except OSError:
    pass

passively creating the output dirname = 
    /Users/aphearin/work/sims/bolplanck/halo_catalogs/halotools_v0p4/a_0.08037


In [8]:
def load_halocat(fname, *colnames):
    pass

In [9]:
def get_column_info(colname, column_info_table):
    idx = np.where(column_info_table['colname'] == colname)[0][0]
    dt = column_info_table['coltype'][idx]
    return idx, dt

In [6]:
get_column_info('vmax', column_info_table)

(16, 'f4')

In [7]:
def build_composite_dt(column_info_table, *colnames):
    dt_list = []
    idx_list = []
    for colname in colnames:
        idx, dt = get_column_info(colname, column_info_table)
        dt_list.append((colname, dt))
        idx_list.append(idx)
    
    idx_sorted = np.argsort(idx_list)
    dt_arr = [tuple(x) for x in np.array(dt_list)[idx_sorted]]
    return np.dtype(dt_arr)

In [8]:
build_composite_dt(column_info_table, 'vmax', 'mvir', 'halo_id')

dtype([('halo_id', '<i8'), ('mvir', '<f4'), ('vmax', '<f4')])

In [9]:
def get_index_list(dt):
    idx_list = []
    for colname in dt.names:
        idx, dt = get_column_info(colname, column_info_table)
        idx_list.append(idx)
    return idx_list

In [10]:
get_index_list(build_composite_dt(column_info_table, 'vmax', 'mvir', 'halo_id'))

[1, 10, 16]

In [12]:
def row_generator(fname, column_info_table, *colnames):
    idx_list = get_index_list(build_composite_dt(column_info_table, *colnames))
    with gzip.open(fname, 'r') as f:
        for raw_line in f:
            if raw_line[0] != '#':
                yield tuple(s for i, s in enumerate(raw_line.strip().split()) if i in idx_list)

In [13]:
d = list(row_generator(halocat_fname, column_info_table, 'vmax', 'mvir', 'halo_id'))

In [14]:
d[0]

('1465370', '7.74900e+08', '43.750000')

In [15]:
arr = np.array(d, dtype=build_composite_dt(column_info_table, 'vmax', 'mvir', 'halo_id'))